In [56]:
from Data import get_min_descriptionsNorm_triples_relations, LOGGER_FILES, PKLS_FILES
from utils.utils import read_cached_array, cache_array, get_logger
from transformers import BertTokenizerFast
from flashtext import KeywordProcessor
from transformers import BertModel
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from utils.model_helpers import get_h_gs, extract_first_embeddings,extract_last_idxs, extract_triples,  merge_triples



Testing compare with extracted triples:

In [31]:
import numpy as np



k = 1000

silver_spans_file = PKLS_FILES["silver_spans"][k]
golden_triples_file = PKLS_FILES["golden_triples"][k]
silver_spans  = read_cached_array(silver_spans_file)
golden_triples_all  = read_cached_array(golden_triples_file)
idxs_not_empty_triples = [idx for idx, (_,v) in enumerate(golden_triples_all.items()) if v["triples"] != []]




tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.,

(array([13, 15, 23, 40]),)

In [113]:
silver_spans["head_end"].shape

torch.Size([3364, 128])

In [40]:

entity_process_idx = idxs_not_empty_triples[5]
entity_process_correct_triples =   golden_triples_all[ list(golden_triples_all.keys())[entity_process_idx]  ]["triples"]

correct_head_start_spans, correct_head_end_spans, correct_tail_start_spans, correct_tail_end_spans = silver_spans["head_start"], silver_spans["head_end"], silver_spans["tail_start"], silver_spans["tail_end"]
entity_correct_head_start_spans, entity_correct_head_end_spans, entity_correct_tail_start_spans, entity_correct_tail_end_spans = correct_head_start_spans[entity_process_idx], correct_head_end_spans[entity_process_idx], correct_tail_start_spans[entity_process_idx], correct_tail_end_spans[entity_process_idx]


In [41]:
batch_size, DESCRIPTION_MAX_LENGTH = correct_head_start_spans.shape

model_head_start_probs = np.zeros((batch_size, DESCRIPTION_MAX_LENGTH))
model_head_end_probs = np.zeros((batch_size, DESCRIPTION_MAX_LENGTH))
model_tail_start_probs = np.zeros((batch_size, DESCRIPTION_MAX_LENGTH))
model_tail_end_probs = np.zeros((batch_size, DESCRIPTION_MAX_LENGTH))








In [87]:

# introduce wrong triples for the process entity model head_start....

for h_idxs, t_idxs in entity_process_correct_triples + [((0,0), (13,13))]:
    model_head_start_probs[entity_process_idx, h_idxs[0]], model_head_end_probs[entity_process_idx, h_idxs[1]], model_tail_start_probs[entity_process_idx, t_idxs[0]], model_tail_end_probs[entity_process_idx, t_idxs[1]] = .9, .9, .9, .9
    
    

In [75]:
entity_process_correct_triples

[((1, 7), (13, 13)),
 ((1, 7), (23, 23)),
 ((1, 7), (40, 40)),
 ((1, 7), (15, 15))]

In [105]:
# Example to understand bce 

import torch
import torch.nn.functional as F

p = torch.tensor([0.9, 0.9, 0.0, 0.0, 0.0])
y = torch.tensor([0.0,  1.0, 0.0, 0.0, 0.0])

loss = F.binary_cross_entropy(p, y)
print(loss.item())


0.48158907890319824


In [101]:
bce = nn.BCELoss()

loss_fwd = 0.0
loss_fwd += bce( torch.tensor(model_head_start_probs).float(), correct_head_start_spans )
loss_fwd += bce( torch.tensor(model_head_end_probs).float(), correct_head_start_spans )
loss_fwd += bce( torch.tensor(model_).float(), correct_head_start_spans )
loss_fwd += bce( torch.tensor(model_head_start_probs).float(), correct_head_start_spans )
loss_fwd.item()


0.17511335015296936

In [12]:


subj_start_probs = np.zeros((batch_size, DESCRIPTION_MAX_LENGTH))
subj_end_probs = np.zeros((batch_size, DESCRIPTION_MAX_LENGTH))


obj_start_probs = np.zeros((batch_size,  DESCRIPTION_MAX_LENGTH))
obj_end_probs = np.zeros((batch_size, DESCRIPTION_MAX_LENGTH))




wrong_triple = ((7,9), (16,18))


for subj_idxs, obj_idxs  in [correct_triples[0],correct_triples[2], wrong_triple]:
    subj_start_probs[sentence_idx, subj_idxs[0]] = 0.9
    subj_end_probs[sentence_idx, subj_idxs[1]] = 0.9
    obj_start_probs[sentence_idx, obj_idxs[0]] = 0.9
    obj_end_probs[sentence_idx, obj_idxs[1]] = 0.9



In [ ]:
import torch

encoded = tokenizer(list(descs.values()), padding="max_length", truncation=True, return_tensors="pt", max_length=128)
input_ids = encoded['input_ids']
attention_mask = encoded['attention_mask']
print(len(input_ids))
model = BertModel.from_pretrained('bert-base-cased')


with torch.no_grad():
    bert_output = model(input_ids=input_ids, attention_mask=attention_mask)
    embeddings = bert_output.last_hidden_state # (batch_size, seq_len ,hidden_size)

embeddings.shape


3364


In [77]:
from utils.model_helpers import extract_first_embeddings, extract_triples , get_h_gs
import torch

model = BertModel.from_pretrained('bert-base-cased')

_, embs = get_h_gs([description], tokenizer,  model, 128)



_, _, f_subj_idxs =  extract_first_embeddings(embs, torch.tensor([subj_start_probs]), torch.tensor([subj_end_probs]), 128, .6)



extracted_triples = extract_triples(f_subj_idxs, obj_start_probs, obj_end_probs, True, 0.6)
extracted_triples



[[((1, 6), 0, (16, 18)),
  ((1, 6), 0, (21, 22)),
  ((1, 6), 0, (38, 39)),
  ((7, 9), 0, (16, 18)),
  ((7, 9), 0, (21, 22)),
  ((7, 9), 0, (38, 39))]]

In [92]:
golden_triples_all = read_cached_array(golden_triples_file)

correct_triples_description_ids = [d_id for d_id in golden_triples_all.keys()]
correct_triples = [b["triples"] for b in golden_triples_all.values()]
len(correct_triples)

530

In [94]:
correct_triples_description_ids[0]

'Q5582517'

In [90]:
correct_triples[0]

[((1, 6), (21, 22)),
 ((1, 6), (28, 29)),
 ((1, 6), (38, 39)),
 ((1, 6), (25, 26)),
 ((1, 6), (24, 26))]

In [81]:
correct_triples = [correct_triples]
batch_size= 1
seq_len = 128

correct_sub_s = torch.zeros(batch_size, seq_len)
correct_sub_e = torch.zeros(batch_size, seq_len)
correct_obj_s = torch.zeros(batch_size, seq_len)
correct_obj_e = torch.zeros(batch_size, seq_len)



for b in range(batch_size):
    for s_idxs, o_idxs in correct_triples[b]:
        for s, e in s_idxs:
            correct_sub_s[b, s] = 1.0
            correct_sub_e[b, e] = 1.0
        for s,e in o_idxs:
            correct_obj_s[b,s]= 1.0
            correct_obj_e[b,e]= 1.0

print("correct_triples: ", correct_triples)
print("correct_sub_s: " , correct_sub_s)


ValueError: too many values to unpack (expected 2)